# Compile the metadata for Hydromet stations in the Pacific North West  
Convert the stations data from text (copied from the website) to accessible JSON.

In [1]:
import json

In [2]:
# path to text file containing usbr_stations_pn_hydromet descriptions from https://www.usbr.gov/pn/hydromet/decod_params.html
path = "stations.txt"

In [3]:
lines = open(path,"r").readlines()
lines = [line.strip(" \n") for line in lines]

# open the stations.json file if it exists else create a new dictionary for the stations
try:
    with open(path.strip(".txt")+".json","r") as f:
        stations_dict = json.load(f)
except:
    stations_dict = {"stations":[]} # initialize the dictionary

i = 0
l = 20

while i<len(lines):
# for i in range(20):
    # line = lines[i].strip(" \n")
    if lines[i].startswith("<A NAME="):
        i += 1
        station_name = lines[i][:4].strip(" ")
        station_description = lines[i][4:].strip(" ")

        stations_dict["stations"].append(station_name.strip(" ")) if station_name.strip(" ") not in stations_dict["stations"] else None

        # add the station to the dictionary if it doesn't exist
        if station_name not in stations_dict:
            stations_dict[station_name] = {}
        stations_dict[station_name]["description"] = station_description # add the station description to the dictionary
        # add the parameters to the dictionary if they don't exist
        if "parameters" not in stations_dict[station_name]:
            stations_dict[station_name]["parameters"] = {}
        
        i += 1
        satellite_id, sattellite_channel = [line.split("=")[1].strip(" ") for line in lines[i].split(",")] # get the satellite id and channel
        stations_dict[station_name]["satellite_id"] = satellite_id # add the satellite id to the dictionary
        stations_dict[station_name]["sattellite_channel"] = sattellite_channel # add the satellite channel to the dictionary
        
        i += 1
        latitude, longitude, elevation = [line.split("=")[1].strip(" ") for line in lines[i].strip(".").split("  ")] # get the latitude, longitude, and elevation
        # add the latitude, longitude, and elevation to the dictionary
        stations_dict[station_name]["latitude"] = latitude
        stations_dict[station_name]["longitude"] = longitude
        stations_dict[station_name]["elevation"] = elevation

        i += 1

        while i < len(lines):
            if lines[i]:
                # get the parameter code and description
                pcode = lines[i][:3].strip(" ")
                pcode_description = lines[i][3:].strip(" ")
                stations_dict[station_name]["parameters"][pcode] = pcode_description # add the parameter to the dictionary

                i += 1
            else:
                i +=1 
                break
    else:
        i += 1

In [4]:
# convert stations dictionary to json file for future use
with open(path.strip(".txt")+".json", "w") as outfile:
    json.dump(stations_dict, outfile)

## Update metadata to include pcodes (parameter codes) from the `pcodes.csv` file

In [3]:
# update json file with parameter codes and descriptions
path = "stations.txt"
stations_dict = json.load(open(path.strip(".txt")+".json","r"))

# path to csv file that contains the station codes and parameter codes
codes_path = "pcodes.csv"
codes = open(codes_path,"r").readlines()
codes = [line.strip(" \n").split(',') for line in codes]

for code in codes:
    station_name = code[0].upper().strip(" ")

    if station_name in stations_dict["stations"]:
        if not "pcodes" in stations_dict[station_name]:
            stations_dict[station_name]["pcodes"] = []
        
        # print(stations_dict[station_name]["pcodes"])
        for pcode in code[1:]:
            if pcode and pcode.strip(' ') not in stations_dict[station_name]["pcodes"]:
                # print(station_name, pcode)
                stations_dict[station_name]["pcodes"].append(pcode.strip(" "))

# convert stations dictionary to json file for future use
with open(path.strip(".txt")+".json", "w") as outfile:
    json.dump(stations_dict, outfile)